This notebook is to analyze the classifier performance over each PD subject regarding the H&Y rating scale.The idea here is get a boxplot over each PD subject in the test set setting the y-axis as the classifier Parkinson output prediction score and the x-axis will be the H&Y puntuation over each PD ID

Notebooks referencia:
1. https://colab.research.google.com/drive/1lOzcGOdi0JA3DGCMZS8vUDH0Y2QTub1-#scrollTo=c6cb21cd
2. https://colab.research.google.com/drive/1T3CmzBZFXVQd7saXVqW9sQiPRP-BtFLO#scrollTo=e47292ba

In [ ]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID";
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [ ]:
import tensorflow as tf
import sklearn
import numpy as np
import pandas as pd
from sklearn.metrics import classification_report, confusion_matrix, multilabel_confusion_matrix, ConfusionMatrixDisplay 
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Input,Dense,GlobalAveragePooling2D,Flatten,concatenate,BatchNormalization, Dropout
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras import regularizers
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import SGD, Adam, RMSprop
from tensorflow.keras.preprocessing import image
from tensorflow import keras
import imageio
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras import regularizers
AUTOTUNE = tf.data.AUTOTUNE
from tqdm import tqdm
from numpy import asarray
import random
from sklearn.metrics import roc_auc_score
from sklearn import preprocessing
from tensorflow import keras
import os
import math
import csv
import seaborn as sns

# <font color='red'>**Helper functions**</font>

In [ ]:
def load_dataframes(task):
    """
    Loads data from three CSV files (training, validation, and test) into pandas DataFrames.

    Returns:
    train_df (pandas.DataFrame): DataFrame containing the training data with two columns, 'path' and 'label'.
    val_df (pandas.DataFrame): DataFrame containing the validation data with two columns, 'path' and 'label'.
    test_df (pandas.DataFrame): DataFrame containing the test data with two columns, 'path' and 'label'.
    """
    # Set the path for the CSV files
    print("working on task: ", task)
    
    gen_path = '/home/Data/Datasets/Parkinson/radiological/PPMI/spect-mri/filtered/' 
    #gen_path = '../../../../../../Datasets/Parkinson/radiological/PPMI/spect-mri/filtered/'
    #synthetic_data = "../imgs_results/full_rois/preprocessed/mri_to_spect/"
    paper_data = "/home/Data/Datasets/Parkinson/radiological/spect_paper/"
    
    if task == "orginal_embc":
        print("working on orginal_embc")
        csv_train = os.path.join(gen_path + 'stratifying_stages/pdControlSpectTRAIN.csv')
        csv_test = os.path.join(gen_path + 'stratifying_stages/pdControlSpectTEST.csv')
        # csv_train = os.path.join(gen_path + 'embc_csvFiles/raw_control_pd_SPECT_fullRois_TRAIN.csv')
        # csv_test = os.path.join(gen_path + 'embc_csvFiles/raw_control_pd_SPECT_fullRois_TEST.csv')
    elif task == "extension_embc":
        csv_train = os.path.join(gen_path + 'pdControlSpectPreprocessed2TRAIN.csv')
        csv_test = os.path.join(gen_path + 'pdControlSpectPreprocessed2TEST.csv')
    elif task == "swedd":
        csv_test = os.path.join(gen_path + 'swedd_SPECT_preprocessed2_TEST.csv')
    else:
        csv_train = os.path.join(paper_data + 'pdControlSpectTRAIN.csv')
        csv_test = os.path.join(paper_data + 'pdControlSpectTEST.csv')
        
    
    if task != "swedd":
        print("reading from : ", csv_train)
        train_df = pd.read_csv(csv_train, header=None)
        train_df.columns = ['path', 'label']

        print("reading from : ", csv_test)
        test_df = pd.read_csv(csv_test, header=None)
        test_df.columns = ['path', 'label']
        
        return train_df, test_df

    else:
        print("reading from : ", csv_test)
        test_df = pd.read_csv(csv_test, header=None)
        test_df.columns = ['path', 'label']
        
        return test_df
        
    
    #========= here for valid set from train set =========
    
    # X = train_df['path']
    # y = train_df['label']

    # # Split the training data into training and validation sets (90-10 split ratio)
    # X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, random_state=14)

    # # Concatenate the training data and their labels to create the training DataFrame
    # train_df = pd.concat([X_train, y_train], axis=1)

    # # Concatenate the validation data and their labels to create the validation DataFrame
    # val_df = pd.concat([X_val, y_val], axis=1)
    
    #========= until here =========   

    # Return the DataFrames containing the loaded data

In [ ]:
# def custom_rescale(img):
#     return (img / 127.5) - 1.0
                                 
def make_generator(df_test, HEIGHT, WIDTH, batch_size):
    
    test_datagen=ImageDataGenerator(rescale=1./255)#ImageDataGenerator(preprocessing_function=custom_rescale)
    
    test_generator=test_datagen.flow_from_dataframe(directory=None,
                                                    dataframe=df_test,
                                                    x_col='path',
                                                    y_col='label',
                                                    batch_size=batch_size,
                                                    seed=42,
                                                    shuffle=False,
                                                    class_mode="categorical",
                                                    target_size=(HEIGHT,WIDTH))

    return test_generator

In [ ]:
def load_model(model_path):
    model = keras.models.load_model(model_path)
    return model

In [ ]:
def predict(label, stage, subject_id, modelo, test_df):
    
    batch = len(test_df)
    test_datagen=ImageDataGenerator(rescale=1./255)
    #Obtiene el número de frames.
    number_of_frames = test_df.Frame
    test_generator=test_datagen.flow_from_dataframe(dataframe=test_df,
                                                  directory=None,
                                                  x_col="path",
                                                  y_col="label",
                                                  batch_size=1,
                                                  seed=42,
                                                  shuffle=False,
                                                  class_mode="categorical",
                                                  target_size=(256, 256))
    pred = modelo.predict_generator(test_generator, steps = batch, verbose=1)
    for k in range(len(pred)):
        pred_k = pred[k]
        writer.writerow([label, stage, subject_id, number_of_frames.iloc[k], pred_k[0], pred_k[1]]) 

In [ ]:
def sort_testing_dataframe(test_df):
    test_df = test_df.sort_values(by='path')
    test_df['Frame'] = (
         test_df.apply(lambda x: int(x.path.split('/')[-1].split('_')[-1][:-4]), axis=1)
         )
    
    test_df['Patient'] = (
         test_df.apply(lambda x: int(x.path.split('/')[-1].split('_')[-3]), axis=1)
         )
    
    return test_df.sort_values(by='Frame')

In [ ]:
def custom_df(df):
        
    scan_info = []
    pred = []
    for i in range(len(df)):
        clase = df.iloc[i]['GT']
        
        if clase == "control":
            clase = "C"
        else:
            clase = "P"
            
        id = df.iloc[i]['PatId']
        stage = df.iloc[i]['H&Y']
        
        to_save = clase + '_' + str(id) + '_' + str(stage)
        scan_info.append(to_save)
        
    scan_set = set(scan_info)
    scan_set = list(scan_set)
    
   
    for i in range(len(df)):
        pd_p = df.iloc[i]['Pd_prob']
        ctrl_p = df.iloc[i]['Ctrl_prob']

        if pd_p > ctrl_p:
            pred.append(0)
        else:
            pred.append(1)

    
    df['Pred'] = pred    
    df['Short_info'] = scan_info 
    
    return df, scan_set

In [ ]:
def plot_probs(df, group):
    # Create a single plot
    fig, ax = plt.subplots(figsize=(30, 10))

    # Box plot
    sns.boxplot(data=df,
            x="H&Y",       # x axis column from data
            y="Pd_prob",          # y axis column from data
            width=0.8,            # The width of the boxes
            color="skyblue",      # Box colour
            linewidth=2,          # Thickness of the box lines
            showfliers=False,     # Stop showing the fliers
            ax=ax)                # Assigning the plot to ax

    # Strip plot
    sns.stripplot(data=df,
                x="H&Y",      # x axis column from data
                y="Pd_prob",         # y axis column from data
                color="crimson",     # Colours the dots
                linewidth=1,         # Dot outline width
                alpha=0.4,           # Makes them transparent
                ax=ax)               # Assigning the plot to ax
    
    # Add a horizontal dotted line at y=0.5
    ax.axhline(y=0.5, color='red', linestyle='--', linewidth=1)

    # Set title and labels for the plot
    ax.set_title("Classifier performance ", fontsize=16)
    ax.set_xlabel("H&Y", fontsize=14)
    ax.set_ylabel("% Pd_prob", fontsize=14)
    ax.tick_params(axis='x', which='both', length=0)  # Remove x-axis ticks and labels

    # Remove vertical gridlines
    ax.grid(axis='y', color='gray', linestyle=':', linewidth=0.5)
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    
    # save the plot as PDF file
    if group == "control":
        plt.savefig("control_plot.pdf", format='pdf')
    else:
        plt.savefig("pd_plot.pdf", format='pdf')
    plt.show()

# <font color='red'>**Main**</font>

**Loading data**

In [ ]:
#load the data
task="swedd"

if task != "swedd":
    train_df, test_df = load_dataframes(task=task)
    print(train_df.groupby('label').count())
    print(test_df.groupby('label').count())
else:
    test_df = load_dataframes(task=task)
    print(test_df.groupby('label').count())  


### Remove the embc baseLine patients

In [ ]:
baseline_csv_file = "/home/Data/Datasets/Parkinson/radiological/PPMI/spect-mri/filtered/embcBaselinev2.csv"
bases_line_df = pd.read_csv(baseline_csv_file)
base_patients = bases_line_df["Pat_id"].unique()
# Convert list to string
ids_to_exclude = [str(id) for id in base_patients]
pattern = '|'.join(ids_to_exclude)

#getting patients that no are in the base for train and test sets
extended_patients_train_df = train_df[~train_df["path"].str.contains(pattern)]
extended_patients_test_df = test_df[~test_df["path"].str.contains(pattern)]

print("amount in train set: ", len(extended_patients_train_df))
print("amount in test set: ", len(extended_patients_test_df))

In [ ]:
train_df = extended_patients_train_df
test_df = extended_patients_test_df

### Until here

In [ ]:
#sorting by case number and slice number for the following situations:
#preprocessing: from 41 to 132
#raw cases: 109 to 180 or 109 to 127----> it needs to be checked
# SPECT with preprocessed2: from 35 to 50

#sorting by case number and slice number
if task!= "swedd":
    #train
    train_df[['case_number', 'slice_number']] = train_df['path'].str.extract(r'_case_(\d+)_slice_(\d+).png').astype(int)
    train_df_v2 = train_df[(train_df['slice_number'] > 35) & (train_df['slice_number'] < 50)]
    train_df_v2.drop('slice_number', axis=1, inplace=True)
    train_df_v2.drop('case_number', axis=1, inplace=True)

    #test
    test_df[['case_number', 'slice_number']] = test_df['path'].str.extract(r'_case_(\d+)_slice_(\d+).png').astype(int)
    test_df_v2 = test_df[(test_df['slice_number'] > 35) & (test_df['slice_number'] < 50)]
    test_df_v2.drop('slice_number', axis=1, inplace=True)
    test_df_v2.drop('case_number', axis=1, inplace=True)
    
    print("lengh of train: {}, lengh of test: {}".format(len(train_df_v2), len(test_df_v2)))
else:
    #test
    test_df[['case_number', 'slice_number']] = test_df['path'].str.extract(r'_case_(\d+)_slice_(\d+).png').astype(int)
    test_df_v2 = test_df[(test_df['slice_number'] > 35) & (test_df['slice_number'] < 50)]
    test_df_v2.drop('slice_number', axis=1, inplace=True)
    test_df_v2.drop('case_number', axis=1, inplace=True)
    
    print("lengh of test: {}".format(len(test_df_v2)))

**From here is for take the PD extra subjects in the extended version**

In [ ]:
curent_df = train_df_v2[train_df_v2['label'] == "parkinson"]
curent_df

**Until here**

In [ ]:
df = sort_testing_dataframe(test_df_v2)#(curent_df)
df = df.reset_index(drop=True)
df

In [ ]:
#adding the H&Y stage 
#txt_file = "/home/Data/Datasets/Parkinson/radiological/PPMI/spect-mri/filtered/stratifying_stages/testPdStages.txt"
if task == "swedd":
    txt_file = "/home/Data/Datasets/Parkinson/radiological/PPMI/spect-mri/filtered/sweddStages.txt"
else:
    txt_file = "/home/Data/Datasets/Parkinson/radiological/PPMI/spect-mri/filtered/extensionPdStagesTest.txt"

# Initialize an empty dictionary
pat_stage_dict = {}

# Open the file using the with statement
with open(txt_file, 'r') as file:
    # Iterate over each line in the file with its line number
    for index, line in enumerate(file):
        # Skip the first line
        if index == 0:
            continue
        # Split the line into id and value based on whitespace
        id, value = line.strip().split()
        # Add the id and value to the dictionary, converting them to the appropriate types
        pat_stage_dict[int(id)] = float(value)

# Print the resulting dictionary
print(pat_stage_dict)


In [ ]:
df["H&Y"] = df["Patient"].map(pat_stage_dict).fillna(0)
subjects = df["Patient"].unique()
print(subjects)
print(len(subjects))

In [ ]:
df

**Loading the model**

**Making predictions**

In [ ]:
print("model loading... ")
name = "MobileNet"
model_path = '../models/embc/classifier/mri_spect_stratified/raw/' + name + '.h5'
model = load_model(model_path)

save_path = '/home/Data/franklin/Doctorado/parkinson/projects/T1-SPECT-PD-translation/imgs_results/full_rois/raw/'
to_write = save_path + 'sweddExtensionSubjectsPreds.csv'

with open( to_write, 'w', newline='') as file:
    writer = csv.writer(file)
    for subject in subjects:
        print(subject)
        single_df = df[df['Patient']==subject]
        single_df = sort_testing_dataframe(single_df)
        label = single_df['label'].iloc[0]
        stage = single_df['H&Y'].iloc[0]
        predict(label, stage, subject, model, single_df)

## Loading the csv predictions

In [ ]:
preds_path = "/home/Data/franklin/Doctorado/parkinson/projects/T1-SPECT-PD-translation/imgs_results/full_rois/raw/sweddExtensionSubjectsPreds.csv"
mobilenet_preds_df = pd.read_csv(preds_path)
mobilenet_preds_df.columns = ["GT", "H&Y", "PatId", "Frame", "Ctrl_prob", "Pd_prob"]
mobilenet_df, scan_info = custom_df(mobilenet_preds_df)
mobilenet_df

In [ ]:
print(sorted(scan_info))
print(len(scan_info))

## Getting boxplots for Parkinson and Control populations

In [ ]:
#getting pd and control subdataframes
ctrl_df = mobilenet_df[mobilenet_df["GT"]=="control"]
pd_df = mobilenet_df[mobilenet_df["GT"]=="parkinson"]

In [ ]:
plot_probs(ctrl_df, "control")

In [ ]:
plot_probs(pd_df, "parkinson")

## Getting boxplots for SWEED and Prodromal

In [ ]:
plot_probs(mobilenet_df, "swedd")